In [73]:
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import pytesseract
import re
import math
import tensorflow as tf


global busNum, busLicenseNum
busNum = '2112'
busLicenseNum = '6211'
# img = './Image/cut_img.JPEG'
img = './Image/sample3.jpg'


def OCR(img):
    ## Original Image Load
    box = cv2.imread(img)
    
    ## 배경 Gray로 설정
    gray = cv2.cvtColor(box, cv2.COLOR_RGB2GRAY)

    ## 이미지 사이즈 정규화(확대)
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)

    ## 이미지 블러처리하기
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    ## 이미지 흑백 대조하기, 최적 임계값을 자동으로 추출하는 Otsus 사용
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

    ## 확장을 위한 커널 생성
    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

    ## 글씨 이미지 크기 확장
    dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

    # find contours of regions of interest within license plate
    try:
        contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    except:
        ret_img, contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # sort contours left-to-right
    sorted_contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

    # create copy of gray image
    im2 = gray.copy()

    # create blank string to hold license plate number
    plate_num = ""
        
    try:
        text = pytesseract.image_to_string(im2, config='-c tessedit_char_whitelist=0123456789 --psm 8 --oem 3')
        # clean tesseract text by removing any unwanted blank spaces
        plate_num = re.sub('[\W_]+', '', text)
    except: 
        text = None
            
    if plate_num != None:
        return plate_num[-4:]
    else:
        return False

    
def tfBusNum(store_img):
    for img in store_img:
        try:
            preBusNum = str(OCR(img)).strip()
            print(f'busNum: {busNum}')
            print(f'prediction busNum: {preBusNum}')
            if preBusNum == str(busNum) or preBusNum == str(busLicenseNum):
                return True  
        except:
            continue
    else:
        return False
    
    
OCR(img)

'2112'